In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import re
import shutil
import pickle
import argparse
from pathlib import Path
from glob import glob
from collections import defaultdict

from tqdm import tqdm
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.linear_model
import statsmodels.api as sm
plt.style.use('default')
from IPython.display import display

from scartan.datasets.meta_oai import (side_code_to_str,
                                       prefix_var_to_visit_month,
                                       release_to_visit_month)
from scartan.datasets.oai._assessments import (read_compose_asmts_mri,
                                               preproc_asmts_biomediq,
                                               preproc_asmts_chondrometrics,
                                               read_info_proj_22,
                                               preproc_info_proj_22)
from scartan.various import (bland_altman_plot, cohen_d,
                             cohen_d_var, r2, linreg)

In [ ]:
PATH_META_OAI_IMO = ('/home/egor/Workspace/proj_scartan'
                     '/data/91_OAI_iMorphics_full_meta/meta_dynamic.csv')

PATH_MRI_AS_ROOT = '/home/egor/MedData/OAI_general/MRI_Assessment_ASCII'
PATH_MRI_AS_SQ = Path(PATH_MRI_AS_ROOT, 'Semi-Quant')
PATH_MRI_AS_Q = Path(PATH_MRI_AS_ROOT, 'Quant')

fpaths_mri_as_biomediq = [*sorted(Path(PATH_MRI_AS_Q).glob('kmri_fnih_qcart_biomediq??.txt'))]
fpaths_mri_as_chondr = [*sorted(Path(PATH_MRI_AS_Q).glob('kmri_qcart_eckstein??.txt'))]

fpath_proj_22_info = '/home/egor/MedData/OAI_general/OAI_CompleteData_ASCII/Clinical_FNIH.txt'

In [ ]:
def add_suffix_sel(df, suffix, include=None, exclude=None):
    if include is None:
        include = df.columns
    if exclude is None:
        exclude = []
    
    cols_pre = df.columns
    cols_post = [str(c) + suffix if ((c in include) and (c not in exclude)) else c
                 for c in cols_pre]
    out = df.copy()
    out.columns = cols_post
    return out

## Read assessment from OAI

In [ ]:
print('-- Project 22 info')
df_proj_22 = read_info_proj_22(fpath_proj_22_info)
df_proj_22_proc = preproc_info_proj_22(df_proj_22)

display(df_proj_22.head())
display(df_proj_22_proc.head())
print(len(df_proj_22))
print(len(df_proj_22_proc))

# Standardize the dataframes, select the data from specific projects
selectors = ['patient', 'side', 'prefix_var', 'visit']

print('-- Biomediq')
df_biomediq = read_compose_asmts_mri(fpaths_mri_as_biomediq, verbose=True)
df_biomediq_proc = preproc_asmts_biomediq(df_biomediq, projects=('22', ))
display(df_biomediq_proc.head())

print('-- Chondrometrics')
df_chondr = read_compose_asmts_mri(fpaths_mri_as_chondr, verbose=True)
df_chondr_proc = preproc_asmts_chondrometrics(df_chondr, projects=('22', ))
display(df_chondr_proc.head())

In [ ]:
asmts_ext_proc = dict()

asmts_ext_proc[("oai_prj_22", "biomediq")] = df_biomediq_proc
asmts_ext_proc[("oai_prj_22", "chondr75n")] = df_chondr_proc

## Exlude subjects from analysis

In [ ]:
# The subjects below were drawn for trainval subset of IMO, thus, used for training of the models
exclusion_overlap = [
    '9369649', '9539084', '9567704', '9587749', '9657464', '9663706', '9693364',
    '9803694', '9867284', '9947240', '9992358', '9993650', '9993833',
]

# The subjects below do not have any of 00m, 12m(!), 24m assessments
exclusion_incomplete = [
    '9065272', '9071463', '9120059', '9135342', '9187064', '9233578', '9338479',
    '9353017', '9444196', '9451499', '9475582', '9489123', '9504935', '9539368',
    '9571631', '9712471', '9811840', '9904574', '9910719', '9936312'
]

for k in (("oai_prj_22", "biomediq"), ("oai_prj_22", "chondr75n")):
    print(k)
    t = asmts_ext_proc[k]
    print("- initial: ", len(t))
    t = t[~t["patient"].isin(exclusion_incomplete)]
    t = t[~t["patient"].isin(exclusion_overlap)]
    print("- after exlcusion: ", len(t))
    asmts_ext_proc[k] = t

## Age, sex, BMI statistics

In [ ]:
def data_stats(df):
    gb_df = df.copy()
    
    print("all")
    tmp_all = gb_df
    print("number: ", len(tmp_all))
    display(tmp_all.loc[:, ["age", "BMI"]].describe())
    unique, counts = np.unique(tmp_all["KL"].values, return_counts=True)
    print("KL: ", dict(zip(unique, counts)))
    
    print("male")
    tmp_male = gb_df[gb_df["sex"] == "MALE"]
    print("number: ", len(tmp_male))
    display(tmp_male.loc[:, ["age", "BMI"]].describe())
    unique, counts = np.unique(tmp_male["KL"].values, return_counts=True)
    print("KL: ", dict(zip(unique, counts)))
    
    print("female")
    tmp_female = gb_df[gb_df["sex"] == "FEMALE"]
    print("number: ", len(tmp_female))
    display(tmp_female.loc[:, ["age", "BMI"]].describe())
    unique, counts = np.unique(tmp_female["KL"].values, return_counts=True)
    print("KL: ", dict(zip(unique, counts)))

In [ ]:
# age, sex
tmp_p = Path("/home/egor/MedData/OAI_general/OAI_CompleteData_ASCII/MeasInventory.csv")
df_tmp0 = pd.read_csv(tmp_p, sep=',', index_col=False)

df_tmp0 = df_tmp0.rename({
    'id': 'patient',
    'V00AGE': 'age',
    'P02SEX': 'sex',
}, axis=1)
df_tmp0.loc[:, 'sex'] = df_tmp0['sex'].replace(
    {'1: Male': 'MALE', '2: Female': 'FEMALE'})
df_tmp0 = df_tmp0.astype({
    'patient': str,
    'age': int,
    'sex': str,
})
df_tmp0 = df_tmp0[["patient", "age", "sex"]]

# BMI
tmp_p = Path("/home/egor/MedData/OAI_general/OAI_CompleteData_ASCII/AllClinical00.txt")
df_tmp1 = pd.read_csv(tmp_p, sep='|', index_col=False)

df_tmp1 = df_tmp1.rename({
    'ID': 'patient',
    'P01BMI': 'BMI',
}, axis=1)
df_tmp1 = df_tmp1.astype({
    'patient': str,
    'BMI': float,
})
df_tmp1 = df_tmp1[["patient", "BMI"]]

In [ ]:
# iMorphics
tmp_p = Path("/home/egor/Workspace/proj_scartan/data/91_OAI_iMorphics_full_meta/meta_base.csv")
df_tmp3 = pd.read_csv(tmp_p)
df_tmp3 = df_tmp3.astype({
    'patient': str,
    'KL': int,
})
df_tmp3 = df_tmp3.sort_values(by=["patient", "prefix_var"], axis=0).drop_duplicates("patient", keep='first')
df_tmp3 = df_tmp3[["patient", "KL"]]

df_t = df_tmp3.copy()
df_t = df_t.merge(df_tmp0, how="inner", on="patient")
df_t = df_t.merge(df_tmp1, how="inner", on="patient")

print(len(df_t))
display(df_t)

data_stats(df_t)

In [ ]:
# iMorphics
tmp_p = Path("/home/egor/Workspace/proj_scartan/data/91_OAI_iMorphics_full_meta/meta_base.csv")
df_tmp3 = pd.read_csv(tmp_p)
df_tmp3 = df_tmp3.astype({
    'patient': str,
    'KL': int,
})
df_tmp3 = df_tmp3.sort_values(by=["patient", "prefix_var"], axis=0).drop_duplicates(["patient", "prefix_var"])
df_tmp3 = df_tmp3[["patient", "KL"]]

df_t = df_tmp3.copy()
df_t = df_t.merge(df_tmp0, how="inner", on="patient")
df_t = df_t.merge(df_tmp1, how="inner", on="patient")

print(len(df_t))
display(df_t)

data_stats(df_t)

In [ ]:
# FNIH Biomarkers
tmp_p = Path("/home/egor/Workspace/proj_scartan/data/61_OAI_project_22_full_meta/meta_base.csv")
df_tmp4 = pd.read_csv(tmp_p)
df_tmp4 = df_tmp4.astype({
    'patient': str,
    'KL': int,
})
df_tmp4 = df_tmp4.sort_values(by=["patient", "prefix_var"], axis=0).drop_duplicates("patient")
df_tmp4 = df_tmp4[["patient", "KL"]]

df_t = df_tmp4.copy()
df_t = df_t.merge(df_tmp0, how="inner", on="patient")
df_t = df_t.merge(df_tmp1, how="inner", on="patient")

df_t = df_t.merge(df_proj_22_proc[["patient", "GROUPTYPE"]], how="inner", on="patient")

print("- initial: ", len(df_t))
df_t = df_t[~df_t["patient"].isin(exclusion_incomplete)]
df_t = df_t[~df_t["patient"].isin(exclusion_overlap)]
print("- after exlcusion: ", len(df_t))

display(df_t)

data_stats(df_t)

for group in pd.unique(df_t["GROUPTYPE"]):
    print(f"---- Group: {group} ----")
    g = df_t[df_t["GROUPTYPE"] == group]
    data_stats(g)

## Read our assessments

In [ ]:
def read_custom_asmts(path_cache):
    with open(path_cache, 'rb') as f:
        d = pickle.load(f)
    df = pd.DataFrame.from_dict(d)
    return df

In [ ]:
def preproc_asmts_ours_imo(df):
    """ """   
    df_tmp = df.copy()

    df_tmp.loc[:, 'visit'] = df_tmp['prefix_var'].apply(
        lambda p: prefix_var_to_visit_month[p])
    
    # Assessments
    # - volume
    k = 'volume_total_pred'
    df_tmp.loc[:, 'F.VC'] = df_tmp[k].apply(lambda x: x[1])
    df_tmp.loc[:, 'LT.VC'] = df_tmp[k].apply(lambda x: x[2])
    df_tmp.loc[:, 'MT.VC'] = df_tmp[k].apply(lambda x: x[3])
    df_tmp.loc[:, 'P.VC'] = df_tmp[k].apply(lambda x: x[4])
    df_tmp.loc[:, 'LM.VC'] = df_tmp[k].apply(lambda x: x[5])
    df_tmp.loc[:, 'MM.VC'] = df_tmp[k].apply(lambda x: x[6])
    
    # - thickness
    k = 'thick_local_mean_pred'
#     k = 'dist_transf_mean_pred'
    if k in df_tmp.columns:
        df_tmp.loc[:, 'F.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[1])
        df_tmp.loc[:, 'LT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[2])
        df_tmp.loc[:, 'MT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[3])
        df_tmp.loc[:, 'P.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[4])
        df_tmp.loc[:, 'LM.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[5])
        df_tmp.loc[:, 'MM.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[6])
    
    selection = [
        'F.VC', 'LT.VC', 'MT.VC', 'P.VC', 'LM.VC', 'MM.VC',
        'F.ThCcAB.MEAN', 'LT.ThCcAB.MEAN', 'MT.ThCcAB.MEAN',
        'P.ThCcAB.MEAN', 'LM.ThCcAB.MEAN', 'MM.ThCcAB.MEAN',
    ]

    df_tmp = df_tmp[['patient', 'side', 'KL',
                     'prefix_var', 'visit', 'release',
                     *selection]]
    return df_tmp

In [ ]:
def preproc_asmts_ours_biomediq(df):
    """ """   
    df_tmp = df.copy()

    df_tmp.loc[:, 'visit'] = df_tmp['prefix_var'].apply(
        lambda p: prefix_var_to_visit_month[p])

    # Assessments
    # - volume
    k = 'volume_total_pred'
    df_tmp.loc[:, 'F.VC'] = df_tmp[k].apply(lambda x: x[1] + x[2])
    df_tmp.loc[:, 'MF.VC'] = df_tmp[k].apply(lambda x: x[1])
    df_tmp.loc[:, 'LF.VC'] = df_tmp[k].apply(lambda x: x[2])
    df_tmp.loc[:, 'T.VC'] = df_tmp[k].apply(lambda x: x[3] + x[4])
    df_tmp.loc[:, 'MT.VC'] = df_tmp[k].apply(lambda x: x[3])
    df_tmp.loc[:, 'LT.VC'] = df_tmp[k].apply(lambda x: x[4])
    df_tmp.loc[:, 'P.VC'] = df_tmp[k].apply(lambda x: x[5])
    df_tmp.loc[:, 'M.VC'] = df_tmp[k].apply(lambda x: x[6] + x[7])
    df_tmp.loc[:, 'MM.VC'] = df_tmp[k].apply(lambda x: x[6])
    df_tmp.loc[:, 'LM.VC'] = df_tmp[k].apply(lambda x: x[7])

    selection = [
        'F.VC', 'MF.VC', 'LF.VC',
        'T.VC', 'MT.VC', 'LT.VC',
        'P.VC',
        'M.VC', 'MM.VC', 'LM.VC',
    ]

    df_tmp = df_tmp[['patient', 'side', 'KL', 'prefix_var', 'visit', 'release',
                     *selection]]
    return df_tmp

In [ ]:
def preproc_asmts_ours_chondr75n(df):
    """ """   
    df_tmp = df.copy()

    df_tmp.loc[:, 'visit'] = df_tmp['prefix_var'].apply(
        lambda p: prefix_var_to_visit_month[p])

    # Assessments
    # - volume
    k = 'volume_total_pred'
    df_tmp.loc[:, 'F.VC'] = df_tmp[k].apply(lambda x: np.sum(x[1:11]))
    df_tmp.loc[:, 'LF.VC'] = df_tmp[k].apply(lambda x: np.sum(x[1:6]))
    df_tmp.loc[:, 'cLF.VC'] = df_tmp[k].apply(lambda x: np.sum(x[2:5]))
    df_tmp.loc[:, 'MF.VC'] = df_tmp[k].apply(lambda x: np.sum(x[6:11]))
    df_tmp.loc[:, 'cMF.VC'] = df_tmp[k].apply(lambda x: np.sum(x[7:10]))
    df_tmp.loc[:, 'tLF.VC'] = df_tmp[k].apply(lambda x: x[1])
    df_tmp.loc[:, 'ccLF.VC'] = df_tmp[k].apply(lambda x: x[2])
    df_tmp.loc[:, 'ecLF.VC'] = df_tmp[k].apply(lambda x: x[3])
    df_tmp.loc[:, 'icLF.VC'] = df_tmp[k].apply(lambda x: x[4])
    df_tmp.loc[:, 'pLF.VC'] = df_tmp[k].apply(lambda x: x[5])
    df_tmp.loc[:, 'tMF.VC'] = df_tmp[k].apply(lambda x: x[6])
    df_tmp.loc[:, 'ccMF.VC'] = df_tmp[k].apply(lambda x: x[7])
    df_tmp.loc[:, 'ecMF.VC'] = df_tmp[k].apply(lambda x: x[8])
    df_tmp.loc[:, 'icMF.VC'] = df_tmp[k].apply(lambda x: x[9])
    df_tmp.loc[:, 'pMF.VC'] = df_tmp[k].apply(lambda x: x[10])
    
    df_tmp.loc[:, 'T.VC'] = df_tmp[k].apply(lambda x: np.sum(x[11:21]))
    df_tmp.loc[:, 'LT.VC'] = df_tmp[k].apply(lambda x: np.sum(x[11:16]))
    df_tmp.loc[:, 'MT.VC'] = df_tmp[k].apply(lambda x: np.sum(x[16:21]))
    df_tmp.loc[:, 'aLT.VC'] = df_tmp[k].apply(lambda x: x[11])
    df_tmp.loc[:, 'pLT.VC'] = df_tmp[k].apply(lambda x: x[12])
    df_tmp.loc[:, 'cLT.VC'] = df_tmp[k].apply(lambda x: x[13])
    df_tmp.loc[:, 'eLT.VC'] = df_tmp[k].apply(lambda x: x[14])
    df_tmp.loc[:, 'iLT.VC'] = df_tmp[k].apply(lambda x: x[15])
    df_tmp.loc[:, 'aMT.VC'] = df_tmp[k].apply(lambda x: x[16])
    df_tmp.loc[:, 'pMT.VC'] = df_tmp[k].apply(lambda x: x[17])
    df_tmp.loc[:, 'cMT.VC'] = df_tmp[k].apply(lambda x: x[18])
    df_tmp.loc[:, 'eMT.VC'] = df_tmp[k].apply(lambda x: x[19])
    df_tmp.loc[:, 'iMT.VC'] = df_tmp[k].apply(lambda x: x[20])
    
    df_tmp.loc[:, 'P.VC'] = df_tmp[k].apply(lambda x: x[21])
    df_tmp.loc[:, 'M.VC'] = df_tmp[k].apply(lambda x: x[22] + x[23])
    df_tmp.loc[:, 'LM.VC'] = df_tmp[k].apply(lambda x: x[22])
    df_tmp.loc[:, 'MM.VC'] = df_tmp[k].apply(lambda x: x[23])
    
    # - thickness
    k = 'thick_local_mean_pred'
#     k = 'dist_transf_mean_pred'
    if k in df_tmp.columns:
        df_tmp.loc[:, 'tLF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[1])
        df_tmp.loc[:, 'ccLF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[2])
        df_tmp.loc[:, 'ecLF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[3])
        df_tmp.loc[:, 'icLF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[4])
        df_tmp.loc[:, 'pLF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[5])
        df_tmp.loc[:, 'tMF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[6])
        df_tmp.loc[:, 'ccMF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[7])
        df_tmp.loc[:, 'ecMF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[8])
        df_tmp.loc[:, 'icMF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[9])
        df_tmp.loc[:, 'pMF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[10])

        df_tmp.loc[:, 'aLT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[11])
        df_tmp.loc[:, 'pLT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[12])
        df_tmp.loc[:, 'cLT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[13])
        df_tmp.loc[:, 'eLT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[14])
        df_tmp.loc[:, 'iLT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[15])
        df_tmp.loc[:, 'aMT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[16])
        df_tmp.loc[:, 'pMT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[17])
        df_tmp.loc[:, 'cMT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[18])
        df_tmp.loc[:, 'eMT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[19])
        df_tmp.loc[:, 'iMT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[20])

    k = 'thick_local_merged_mean_pred'
#     k = 'dist_transf_merged_mean_pred'
    if k in df_tmp.columns:
        df_tmp.loc[:, 'cLF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[0])
        df_tmp.loc[:, 'cMF.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[1])
        df_tmp.loc[:, 'LT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[2])
        df_tmp.loc[:, 'MT.ThCcAB.MEAN'] = df_tmp[k].apply(lambda x: x[3])
        df_tmp.loc[:, 'cLFTC.ThCcAB.MEAN'] = \
            df_tmp.loc[:, 'ccLF.ThCcAB.MEAN'] + df_tmp.loc[:, 'cLT.ThCcAB.MEAN']
        df_tmp.loc[:, 'cMFTC.ThCcAB.MEAN'] = \
            df_tmp.loc[:, 'ccMF.ThCcAB.MEAN'] + df_tmp.loc[:, 'cMT.ThCcAB.MEAN']
        df_tmp.loc[:, 'LFTC.ThCcAB.MEAN'] = \
            df_tmp.loc[:, 'cLF.ThCcAB.MEAN'] + df_tmp.loc[:, 'LT.ThCcAB.MEAN']
        df_tmp.loc[:, 'MFTC.ThCcAB.MEAN'] = \
            df_tmp.loc[:, 'cMF.ThCcAB.MEAN'] + df_tmp.loc[:, 'MT.ThCcAB.MEAN']

    # NOTICE: Rename cAB to tAB for code convenience in comparing to Chondrometrics
    for c_from in df_tmp.columns:
        if '.ThCcAB.' in c_from:
            c_to = c_from.replace('.ThCcAB.', '.ThCtAB.')
            df_tmp.loc[:, c_to] = df_tmp[c_from]
    
    selection = [
        'F.VC', 'LF.VC', 'MF.VC', 'cLF.VC', 'cMF.VC',
        'tLF.VC', 'ccLF.VC', 'ecLF.VC', 'icLF.VC', 'pLF.VC',
        'tMF.VC', 'ccMF.VC', 'ecMF.VC', 'icMF.VC', 'pMF.VC',
        'T.VC', 'LT.VC', 'MT.VC',
        'aLT.VC', 'pLT.VC', 'cLT.VC', 'eLT.VC', 'iLT.VC',
        'aMT.VC', 'pMT.VC', 'cMT.VC', 'eMT.VC', 'iMT.VC',
        'P.VC', 'M.VC', 'LM.VC', 'MM.VC',
        
        'aLT.ThCcAB.MEAN', 'iLT.ThCcAB.MEAN', 'cLT.ThCcAB.MEAN', 'pLT.ThCcAB.MEAN', 'eLT.ThCcAB.MEAN',
        'aMT.ThCcAB.MEAN', 'iMT.ThCcAB.MEAN', 'cMT.ThCcAB.MEAN', 'pMT.ThCcAB.MEAN', 'eMT.ThCcAB.MEAN',
        
        'tLF.ThCcAB.MEAN', 'pLF.ThCcAB.MEAN', 'ecLF.ThCcAB.MEAN', 'ccLF.ThCcAB.MEAN', 'icLF.ThCcAB.MEAN',
        'tMF.ThCcAB.MEAN', 'pMF.ThCcAB.MEAN', 'ecMF.ThCcAB.MEAN', 'ccMF.ThCcAB.MEAN', 'icMF.ThCcAB.MEAN',
        
        'cLF.ThCcAB.MEAN', 'LT.ThCcAB.MEAN', 'cLFTC.ThCcAB.MEAN', 'LFTC.ThCcAB.MEAN',
        'cMF.ThCcAB.MEAN', 'MT.ThCcAB.MEAN', 'cMFTC.ThCcAB.MEAN', 'MFTC.ThCcAB.MEAN',
        'aLT.ThCtAB.MEAN', 'iLT.ThCtAB.MEAN', 'cLT.ThCtAB.MEAN', 'pLT.ThCtAB.MEAN', 'eLT.ThCtAB.MEAN',
        'aMT.ThCtAB.MEAN', 'iMT.ThCtAB.MEAN', 'cMT.ThCtAB.MEAN', 'pMT.ThCtAB.MEAN', 'eMT.ThCtAB.MEAN',
        
        'tLF.ThCtAB.MEAN', 'pLF.ThCtAB.MEAN', 'ecLF.ThCtAB.MEAN', 'ccLF.ThCtAB.MEAN', 'icLF.ThCtAB.MEAN',
        'tMF.ThCtAB.MEAN', 'pMF.ThCtAB.MEAN', 'ecMF.ThCtAB.MEAN', 'ccMF.ThCtAB.MEAN', 'icMF.ThCtAB.MEAN',
        
        'cLF.ThCtAB.MEAN', 'LT.ThCtAB.MEAN', 'cLFTC.ThCtAB.MEAN', 'LFTC.ThCtAB.MEAN',
        'cMF.ThCtAB.MEAN', 'MT.ThCtAB.MEAN', 'cMFTC.ThCtAB.MEAN', 'MFTC.ThCtAB.MEAN',
    ]

    df_tmp = df_tmp[['patient', 'side', 'KL', 'prefix_var', 'visit', 'release',
                     *selection]]
    return df_tmp

In [ ]:
asmts_ours = dict()  # experiment , dataset , atlas , kind

# Segmentation SotA
asmts_ours[('20190720_0001', 'oai_imo', 'imo', 'paired')] = None

# Comparison SotA
asmts_ours[('20190720_0001', 'oai_prj_22', 'biomediq', 'single')] = None
asmts_ours[('20190720_0001', 'oai_prj_22', 'chondr75n', 'single')] = None

In [ ]:
for k in asmts_ours.keys():
    experiment, dataset, atlas, kind = k
    
    path_results = '/home/egor/Workspace/proj_scartan/results/'
    
    path_cache = Path(
        path_results,
        experiment,
        f'logs_{dataset}_test',
        f'cache_{dataset}_test_{atlas}_volumew_{kind}.pkl')

    asmts_ours[k] = read_custom_asmts(path_cache)

# iMoprhics -- Analysis of segmentation

In [ ]:
# Preprocess. Segmentation assessment
asmts_pred_proc = dict()
asmts_true_proc = dict()

k = ('20190720_0001', 'oai_imo', 'imo', 'paired')
df_tmp = asmts_ours[k].copy()
asmts_pred_proc[k] = preproc_asmts_ours_imo(df_tmp)
# preproc_asmts_... is implemented for ..._pred only
df_tmp = asmts_ours[k].copy()
df_tmp.loc[:, 'volume_total_pred'] = df_tmp['volume_total_true']
df_tmp.loc[:, 'thick_local_mean_pred'] = df_tmp['thick_local_mean_true']
df_tmp.loc[:, 'dist_transf_mean_pred'] = df_tmp['dist_transf_mean_true']
asmts_true_proc[k] = preproc_asmts_ours_imo(df_tmp)

k = ('20190720_0001', 'oai_imo', 'chondr75n', 'paired')
df_tmp = asmts_ours[k].copy()
asmts_pred_proc[k] = preproc_asmts_ours_chondr75n(df_tmp)
# preproc_asmts_... is implemented for ..._pred only
df_tmp = asmts_ours[k].copy()
df_tmp.loc[:, 'volume_total_pred'] = df_tmp['volume_total_true']
df_tmp.loc[:, 'thick_local_mean_pred'] = df_tmp['thick_local_mean_true']
df_tmp.loc[:, 'dist_transf_mean_pred'] = df_tmp['dist_transf_mean_true']
asmts_true_proc[k] = preproc_asmts_ours_chondr75n(df_tmp)

In [ ]:
# Analyze our segmentation in relation to the reference. OAI iMorphics
path_out = Path('/home/egor/Workspace/proj_scartan/results/t/0_segm_imo')
path_out.mkdir(exist_ok=True)

# k = ('20190720_0001', 'oai_imo', 'segm', 'paired')
k = ('20190720_0001', 'oai_imo', 'imo', 'paired')
# k = ('20190720_0001', 'oai_imo', 'chondr75n', 'paired')
# k = ('20190720_0001', 'oai_prj_22', 'chondr75n', 'single')

feature_codes = (
    ## segm
#     'F.VC', 'T.VC', 'P.VC', 'M.VC',
#     'F.ThCcAB.MEAN', 'T.ThCcAB.MEAN', 'P.ThCcAB.MEAN', 'M.ThCcAB.MEAN',
    
    ## imo
    'F.VC', 'LT.VC', 'MT.VC', 'P.VC', 'LM.VC', 'MM.VC',
    'F.ThCcAB.MEAN', 'LT.ThCcAB.MEAN', 'MT.ThCcAB.MEAN',
    'P.ThCcAB.MEAN', 'LM.ThCcAB.MEAN', 'MM.ThCcAB.MEAN',
    
    ## pseudo-chondr
#     'tLF.VC', 'ecLF.VC', 'ccLF.VC', 'icLF.VC', 'pLF.VC',
#     'tMF.VC', 'ecMF.VC', 'ccMF.VC', 'icMF.VC', 'pMF.VC',
#     'aLT.VC', 'pLT.VC', 'cLT.VC', 'eLT.VC', 'iLT.VC',
#     'aMT.VC', 'pMT.VC', 'cMT.VC', 'eMT.VC', 'iMT.VC',
    
#     'tLF.ThCcAB.MEAN', 'ecLF.ThCcAB.MEAN', 'ccLF.ThCcAB.MEAN', 'icLF.ThCcAB.MEAN', 'pLF.ThCcAB.MEAN',
#     'tMF.ThCcAB.MEAN', 'ecMF.ThCcAB.MEAN', 'ccMF.ThCcAB.MEAN', 'icMF.ThCcAB.MEAN', 'pMF.ThCcAB.MEAN',
    
#     'aLT.ThCcAB.MEAN', 'iLT.ThCcAB.MEAN', 'cLT.ThCcAB.MEAN', 'pLT.ThCcAB.MEAN', 'eLT.ThCcAB.MEAN',
#     'aMT.ThCcAB.MEAN', 'iMT.ThCcAB.MEAN', 'cMT.ThCcAB.MEAN', 'pMT.ThCcAB.MEAN', 'eMT.ThCcAB.MEAN',
    
    ## chondr
#     'cLF.VC', 'cMF.VC',
#     'LT.VC', 'MT.VC',
#     'cLF.ThCcAB.MEAN', 'ecLF.ThCcAB.MEAN', 'ccLF.ThCcAB.MEAN', 'icLF.ThCcAB.MEAN',
#     'cMF.ThCcAB.MEAN', 'ecMF.ThCcAB.MEAN', 'ccMF.ThCcAB.MEAN', 'icMF.ThCcAB.MEAN',
#     'aLT.ThCcAB.MEAN', 'iLT.ThCcAB.MEAN', 'cLT.ThCcAB.MEAN', 'pLT.ThCcAB.MEAN', 'eLT.ThCcAB.MEAN',
#     'aMT.ThCcAB.MEAN', 'iMT.ThCcAB.MEAN', 'cMT.ThCcAB.MEAN', 'pMT.ThCcAB.MEAN', 'eMT.ThCcAB.MEAN',
)
for feature_code in feature_codes:

    df_plot = pd.DataFrame.from_dict({
        'pred': asmts_pred_proc[k][feature_code],
        'true': asmts_true_proc[k][feature_code],
        'side': asmts_pred_proc[k]['side'],
    })
    
    plot_kws = {'kind': 'reg',
                'stat_func': r2, 'height': 7}
    g = sns.jointplot(x='pred', y='true', data=df_plot, **plot_kws,
                      joint_kws={'robust': True})
    
    if '.VC' in feature_code:
        g.set_axis_labels("Ours (mm^3)", "Reference (mm^3)")
    elif '.ThC' in feature_code:
        g.set_axis_labels("Ours (mm)", "Reference (mm)")
    else:
        print('Unhandled feature code')
        break
        
#     plt.axis('equal')
    plt.title(feature_code)

    fname_out = f'{feature_code}_joint.png'
    fpath_out = Path(path_out, fname_out)
    plt.tight_layout()
#     plt.show()
    plt.savefig(fpath_out)
    plt.close()

    fig, axes = plt.subplots(1)
    bland_altman_plot(df_plot['pred'], df_plot['true'],
                      sd_limit=1.96,
                      ax=axes,
                      scatter_kws={'s': 3},
                      mean_line_kws=None,
                      limit_lines_kws=None)
#     plt.axis('equal')
    plt.title(feature_code)

    fname_out = f'{feature_code}_bland_altman.png'
    fpath_out = Path(path_out, fname_out)
    plt.tight_layout()
#     plt.show()
    plt.savefig(fpath_out)
    plt.close()
    
    m1 = df_plot['pred']
    m2 = df_plot['true']
    means = np.mean([m1, m2], axis=0)
    diffs = m1 - m2
    mean_diff = np.mean(diffs)
    std_diff = np.std(diffs, axis=0)
    
    print(feature_code,
          r2(df_plot['pred'], df_plot['true']),
          np.round(-1.96 * std_diff + mean_diff, 3),
          np.round(mean_diff, 3),
          np.round(1.96 * std_diff + mean_diff, 3))

# FNIH Biom-s -- Comparison to other methods

In [ ]:
# Preprocess. Compartmentalization
selectors = ['patient', 'side', 'prefix_var', 'visit']

asmts_ours_proc = dict()

k = ('20190720_0001', 'oai_prj_22', 'biomediq', 'single')
asmts_ours_proc[k] = preproc_asmts_ours_biomediq(asmts_ours[k])

k = ('20190720_0001', 'oai_prj_22', 'chondr75n', 'single')
asmts_ours_proc[k] = preproc_asmts_ours_chondr75n(asmts_ours[k])

In [ ]:
# INFO: Temporary! For extension of analysis only
k_ext = ('oai_prj_22', 'chondr75n')
asmts_ext_proc[k_ext]

for field_i, field_o in (
    ("LT.ThCcAB.MEAN", "LT.ThCtAB.MEAN"),
    ("MT.ThCcAB.MEAN", "MT.ThCtAB.MEAN"),
    ("cLF.ThCcAB.MEAN", "cLF.ThCtAB.MEAN"),
    ("cMF.ThCcAB.MEAN", "cMF.ThCtAB.MEAN"),
):
    asmts_ext_proc[k_ext].loc[:, field_o] = asmts_ext_proc[k_ext][field_i]

In [ ]:
# Extract common measurements
selectors = ['patient', 'side', 'prefix_var', 'visit']

asmts_compare = dict()

k_ours = ('20190720_0001', 'oai_prj_22', 'biomediq', 'single')
k_ext = ('oai_prj_22', 'biomediq')
asmts_compare[('ours', 'biomediq')] = asmts_ours_proc[k_ours].merge(
    asmts_ext_proc[k_ext], on=selectors,
    how='inner', suffixes=('_ours', '_biomediq'))

k_ours = ('20190720_0001', 'oai_prj_22', 'chondr75n', 'single')
k_ext = ('oai_prj_22', 'chondr75n')
asmts_compare[('ours', 'chondr75n')] = asmts_ours_proc[k_ours].merge(
    asmts_ext_proc[k_ext], on=selectors,
    how='inner', suffixes=('_ours', '_chondr'))

In [ ]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [ ]:
# Biomediq
path_out = Path('/home/egor/Workspace/proj_scartan/results/t/0j_biomediq_segm')
path_out.mkdir(exist_ok=True, parents=True)

feature_codes = (
    'F.VC', 'LF.VC', 'MF.VC', 
    'LT.VC', 'MT.VC',
    'P.VC',
    'LM.VC', 'MM.VC'
)

matplotlib.rcParams.update({'font.size': 14})

df_tmp = asmts_compare[('ours', 'biomediq')]

for feature_code in feature_codes:    

    df_plot = pd.DataFrame.from_dict({
        'ours': df_tmp[f'{feature_code}_ours'],
        'biomediq': df_tmp[f'{feature_code}_biomediq'],
        'side': df_tmp['side'],
    })
    
    if True:
#     if False:
        g = sns.jointplot(x='ours', y='biomediq', data=df_plot,
                          kind='reg', stat_func=r2,
                          height=7)

        if ".VC" in feature_code:
            g.set_axis_labels("Ours (mm^3)", "Biomediq (mm^3)")
        elif ".ThC" in feature_code:
            g.set_axis_labels("Ours (mm)", "Biomediq (mm)")
            
        if ".VC" in feature_code:
            plt.ticklabel_format(style='sci', axis='both', scilimits=(3,3))

        plt.title(feature_code)

        fname_out = f"{feature_code}.png"
        fpath_out = Path(path_out, fname_out)
        plt.tight_layout()
        plt.savefig(fpath_out)

        plt.close()

    if True:
#     if False:
        fig, axes = plt.subplots()
    
        if ".VC" in feature_code:
            xlabel = "Means (mm^3)"
            ylabel = "Difference (mm^3)"
        elif ".ThC" in feature_code:
            xlabel = "Means (mm)"
            ylabel = "Difference (mm)"
            
        bland_altman_plot(df_plot['ours'], df_plot['biomediq'],
                          sd_limit=1.96,
                          ax=axes,
                          scatter_kws={'s': 3},
                          mean_line_kws=None,
                          limit_lines_kws=None,
                          xlabel=xlabel,
                          ylabel=ylabel,
                         )
        
        if ".VC" in feature_code:
            plt.ticklabel_format(style='sci', axis='both', scilimits=(3,3))
        
        plt.title(feature_code.replace("tAB", "").replace(".MEAN", ""))

        fname_out = f'{feature_code}_bland_altman.png'
        fpath_out = Path(path_out, fname_out)
        plt.tight_layout()
        plt.savefig(fpath_out)

    # Textual
    m1 = df_plot['ours']
    m2 = df_plot['biomediq']
    
    means = np.mean([m1, m2], axis=0)
    diffs = m1 - m2
    mean_diff = np.mean(diffs)
    std_diff = np.std(diffs, axis=0)
    
    regr = linreg(m1, m2)
    
    print(
        feature_code, ',',
        np.round(-1.96 * std_diff + mean_diff, 3), ',',
        np.round(mean_diff, 3), ',',
        np.round(1.96 * std_diff + mean_diff, 3), ',',
        f"{np.mean(m2):.3f}",
        f"{regr['r2']:.3f}",
        f"{regr['pvalue']:.3f}",
        f"{np.sqrt(regr['r2']):.3f}",
    )

In [ ]:
# Chondrometrics
path_out = Path('/home/egor/Workspace/proj_scartan/results/t/0j_chondr_segm')
path_out.mkdir(exist_ok=True, parents=True)

# feature_codes = (
#     'cLF.VC', 'LT.VC', 'cMF.VC', 'MT.VC',
    
#     'LFTC.ThCtAB.MEAN', 'cLFTC.ThCtAB.MEAN', 'cLF.ThCtAB.MEAN',
#     'ecLF.ThCtAB.MEAN', 'ccLF.ThCtAB.MEAN', 'icLF.ThCtAB.MEAN',
#     'LT.ThCtAB.MEAN',
#     'aLT.ThCtAB.MEAN', 'eLT.ThCtAB.MEAN', 'cLT.ThCtAB.MEAN', 'iLT.ThCtAB.MEAN', 'pLT.ThCtAB.MEAN', 
    
#     'MFTC.ThCtAB.MEAN', 'cMFTC.ThCtAB.MEAN', 'cMF.ThCtAB.MEAN',
#     'ecMF.ThCtAB.MEAN', 'ccMF.ThCtAB.MEAN', 'icMF.ThCtAB.MEAN',
#     'MT.ThCtAB.MEAN',
#     'aMT.ThCtAB.MEAN', 'eMT.ThCtAB.MEAN', 'cMT.ThCtAB.MEAN', 'iMT.ThCtAB.MEAN', 'pMT.ThCtAB.MEAN',
# )
feature_codes = (
    'cLF.ThCtAB.MEAN', 'LT.ThCtAB.MEAN',
    'cMF.ThCtAB.MEAN', 'MT.ThCtAB.MEAN',
)
# feature_codes = (
#     'LT.VC', 'MT.VC'
# )

matplotlib.rcParams.update({'font.size': 14})

df_tmp = asmts_compare[('ours', 'chondr75n')]

df_tmp.head()

for feature_code in feature_codes:
    df_plot = pd.DataFrame.from_dict({
        'ours': df_tmp[f'{feature_code}_ours'],
        'chondr': df_tmp[f'{feature_code}_chondr'],
        'side': df_tmp['side'],
    })
    
    if True:
#     if False:
        g = sns.jointplot(x='ours', y='chondr', data=df_plot,
                          kind="reg", stat_func=r2,
                          height=7)   

        if ".VC" in feature_code:
            g.set_axis_labels("Ours (mm^3)", "Chondrometrics (mm^3)")
        elif ".ThC" in feature_code:
            g.set_axis_labels("Ours (mm)", "Chondrometrics (mm)")
            
        if ".VC" in feature_code:
            plt.ticklabel_format(style='sci', axis='both', scilimits=(3,3))

        plt.title(feature_code)

        fname_out = f'{feature_code}.png'
        fpath_out = Path(path_out, fname_out)
        plt.tight_layout()
        plt.savefig(fpath_out)
        plt.close()
#         plt.show()
    
    if True:
#     if False:
        fig, axes = plt.subplots(1)
    
        if ".VC" in feature_code:
            xlabel = "Means (mm^3)"
            ylabel = "Difference (mm^3)"
        elif ".ThC" in feature_code:
            xlabel = "Means (mm)"
            ylabel = "Difference (mm)"
    
        bland_altman_plot(df_plot['ours'], df_plot['chondr'],
                          sd_limit=1.96,
                          ax=axes,
                          scatter_kws={'s': 2},
                          mean_line_kws=None,
                          limit_lines_kws=None,
                          xlabel=xlabel,
                          ylabel=ylabel,
                         )
        
        if ".VC" in feature_code:
            plt.ticklabel_format(style='sci', axis='both', scilimits=(3,3))

        plt.title(feature_code.replace("tAB", "").replace(".MEAN", ""))

        fname_out = f'{feature_code}_bland_altman.png'
        fpath_out = Path(path_out, fname_out)
        plt.tight_layout()
        plt.savefig(fpath_out)
        plt.close()
#         plt.show()
    
    # Textual
    m1 = df_plot['ours']
    m2 = df_plot['chondr']
    
    means = np.mean([m1, m2], axis=0)
    diffs = m1 - m2
    mean_diff = np.mean(diffs)
    std_diff = np.std(diffs, axis=0)
    
    regr = linreg(m1, m2)
    
    print(
        feature_code, ',',
        np.round(-1.96 * std_diff + mean_diff, 3), ',',
        np.round(mean_diff, 3), ',',
        np.round(1.96 * std_diff + mean_diff, 3), ',',
        f"{np.mean(m2):.3f}",
        f"{regr['r2']:.3f}",
        f"{regr['pvalue']:.3f}",
        f"{np.sqrt(regr['r2']):.3f}",
    )
    #break

# FNIH Biomarkers -- Comparison of effects

In [ ]:
# Preprocess. Compartmentalization
selectors = ['patient', 'side', 'prefix_var', 'visit']

asmts_ours_proc = dict()

k = ('20190720_0001', 'oai_prj_22', 'biomediq', 'single')
asmts_ours_proc[k] = preproc_asmts_ours_biomediq(asmts_ours[k])

k = ('20190720_0001', 'oai_prj_22', 'chondr75n', 'single')
asmts_ours_proc[k] = preproc_asmts_ours_chondr75n(asmts_ours[k])

In [ ]:
# Extract common measurements
selectors = ['patient', 'side', 'prefix_var', 'visit']

asmts_compare = dict()

k_ours = ('20190720_0001', 'oai_prj_22', 'biomediq', 'single')
k_ext = ('oai_prj_22', 'biomediq')
asmts_compare[('ours', 'biomediq')] = asmts_ours_proc[k_ours].merge(
    asmts_ext_proc[k_ext], on=selectors,
    how='inner', suffixes=('_ours', '_biomediq'))

k_ours = ('20190720_0001', 'oai_prj_22', 'chondr75n', 'single')
k_ext = ('oai_prj_22', 'chondr75n')
asmts_compare[('ours', 'chondr75n')] = asmts_ours_proc[k_ours].merge(
    asmts_ext_proc[k_ext], on=selectors,
    how='inner', suffixes=('_ours', '_chondr'))

In [ ]:
k_ext = ('oai_prj_22', 'chondr75n')
len(asmts_ext_proc[k_ext]["cLF.ThCcAB.MEAN"] == asmts_ext_proc[k_ext]["cLF.ThCtAB.MEAN"])

In [ ]:
# Mixin the project info
selectors = ['patient', 'side']

k = ('ours', 'biomediq')
asmts_compare[k] = asmts_compare[k].merge(
    df_proj_22_proc, on=selectors,
    how='left', suffixes=('', '_proj'))

k = ('ours', 'chondr75n')
asmts_compare[k] = asmts_compare[k].merge(
    df_proj_22_proc, on=selectors,
    how='left', suffixes=('', '_proj'))

In [ ]:
case_code_to_str = {
    1: 'JSL & Pain Prog.',
    2: 'JSL Prog.',
    3: 'Pain Prog.',
    4: 'Non-Prog.',
}

In [ ]:
# Check if some cases are with incomplete assessments
for gn, gdf in df_tmp.groupby(['patient', 'side']):
    if len(gdf) < 3:
        print(f"'{gn[0]}'")
#         display(gdf)

In [ ]:
# Odds ratio

# INFO: Choose one
# analysis = ("biomediq", "")
# analysis = ("chondr", "elastic_lt")
analysis = ("chondr", "elastic_dt")


if analysis[0] == "biomediq":
    k = ('ours', 'biomediq')
elif analysis[0] == "chondr":
    k = ('ours', 'chondr75n')
else:
    raise ValueError()

df_tmp = asmts_compare[k]

if analysis[0] == "biomediq":
    feature_codes = (
        'F.VC', 'LF.VC', 'MF.VC',
        'LT.VC', 'MT.VC',
        'P.VC',
        'LM.VC', 'MM.VC',
    )
elif analysis[0] == "chondr":
    feature_codes = (
    'cLF.VC', 'cMF.VC',
    'LT.VC', 'MT.VC',
    'ecLF.ThCtAB.MEAN', 'ccLF.ThCtAB.MEAN', 'icLF.ThCtAB.MEAN',
    'ecMF.ThCtAB.MEAN', 'ccMF.ThCtAB.MEAN', 'icMF.ThCtAB.MEAN',
    'aLT.ThCtAB.MEAN', 'iLT.ThCtAB.MEAN', 'cLT.ThCtAB.MEAN', 'pLT.ThCtAB.MEAN', 'eLT.ThCtAB.MEAN',
    'aMT.ThCtAB.MEAN', 'iMT.ThCtAB.MEAN', 'cMT.ThCtAB.MEAN', 'pMT.ThCtAB.MEAN', 'eMT.ThCtAB.MEAN',
    'cLF.ThCtAB.MEAN', 'LT.ThCtAB.MEAN', 'cLFTC.ThCtAB.MEAN', 'LFTC.ThCtAB.MEAN',
    'cMF.ThCtAB.MEAN', 'MT.ThCtAB.MEAN', 'cMFTC.ThCtAB.MEAN', 'MFTC.ThCtAB.MEAN',
    
    'LT.ThCcAB.MEAN', 'MT.ThCcAB.MEAN', 'cMF.ThCcAB.MEAN', 'cLF.ThCcAB.MEAN',
)

comparisons = [
    ((4, ), (2, ), '000m', '012m'),
    ((4, ), (2, ), '000m', '024m'),
    ((3, 4), (1, 2), '000m', '012m'),
    ((3, 4), (1, 2), '000m', '024m'),
]

for comparison in comparisons:
    cases_ctr, cases_aff, visit_base, visit_cont = comparison
    
    dict_res = defaultdict(list)
    
    # Select subject groups: control and case, baseline and continuation
    sel_ctr_base = df_tmp[df_tmp['CASE'].isin(cases_ctr) & (df_tmp['visit'] == visit_base)]
    sel_ctr_cont = df_tmp[df_tmp['CASE'].isin(cases_ctr) & (df_tmp['visit'] == visit_cont)]
    sel_aff_base = df_tmp[df_tmp['CASE'].isin(cases_aff) & (df_tmp['visit'] == visit_base)]
    sel_aff_cont = df_tmp[df_tmp['CASE'].isin(cases_aff) & (df_tmp['visit'] == visit_cont)]
    
    for feature_code in feature_codes:
        author_list = ("ours", analysis[0])
        
        for author in author_list:
            if f"{feature_code}_{author}" not in df_tmp.columns:
                continue

            val_ctr_base = np.asarray(sel_ctr_base[f'{feature_code}_{author}'].tolist())
            val_ctr_cont = np.asarray(sel_ctr_cont[f'{feature_code}_{author}'].tolist())
            val_aff_base = np.asarray(sel_aff_base[f'{feature_code}_{author}'].tolist())
            val_aff_cont = np.asarray(sel_aff_cont[f'{feature_code}_{author}'].tolist())

            diff_ctr = val_ctr_base - val_ctr_cont
            diff_aff = val_aff_base - val_aff_cont
            
            X = [*diff_ctr, *diff_aff]
            y_true = [0, ] * len(diff_ctr) + [1, ] * len(diff_aff)
            
            X = np.asarray(X)
            y_true = np.asarray(y_true)
            sel = np.invert(np.isnan(X))
            X = X[sel][..., None]
            y_true = y_true[sel]

            clf = sklearn.linear_model.LogisticRegression(penalty='none',
                                                          class_weight='balanced',
                                                          solver='newton-cg')
            clf.fit(X, y_true)
            y_pred = clf.predict(X)
            
            cm = sklearn.metrics.confusion_matrix(y_true, y_pred)
            odds_ratio, pvalue = stats.fisher_exact(cm)

            if pvalue < 0.05:
                print(feature_code, author)

            dict_res['feature_code'].append(feature_code)
            dict_res['author'].append(author)

            dict_res['odds_ratio'].append(odds_ratio)
            dict_res['pvalue'].append(f"{pvalue:0.3f}")

    df_res = pd.DataFrame.from_dict(dict_res)
    display(df_res)

    path_out = Path(f"/home/egor/Workspace/proj_scartan/results/t/"
                    f"csvs_{analysis[0]}_{analysis[1]}")
    path_out.mkdir(exist_ok=True)
    
    str_ctr = "".join(str(e) for e in cases_ctr)
    str_aff = "".join(str(e) for e in cases_aff)

    df_res.to_csv(Path(
        path_out, f"{str_ctr}vs{str_aff}_{visit_base}-{visit_cont}.csv"))

In [ ]:
# KL-wise profiling of ORs

# Odds ratio

# INFO: Choose one
# analysis = ("biomediq", "")
# analysis = ("chondr", "elastic_lt")
analysis = ("chondr", "elastic_dt")


if analysis[0] == "biomediq":
    k = ('ours', 'biomediq')
elif analysis[0] == "chondr":
    k = ('ours', 'chondr75n')
else:
    raise ValueError()

df_sel_cmp = asmts_compare[k]

if analysis[0] == "biomediq":
    feature_codes = (
        'F.VC', 'LF.VC', 'MF.VC',
        'LT.VC', 'MT.VC',
        'P.VC',
        'LM.VC', 'MM.VC',
    )
elif analysis[0] == "chondr":
    feature_codes = (
    'cLF.VC', 'cMF.VC',
    'LT.VC', 'MT.VC',
    'ecLF.ThCtAB.MEAN', 'ccLF.ThCtAB.MEAN', 'icLF.ThCtAB.MEAN',
    'ecMF.ThCtAB.MEAN', 'ccMF.ThCtAB.MEAN', 'icMF.ThCtAB.MEAN',
    'aLT.ThCtAB.MEAN', 'iLT.ThCtAB.MEAN', 'cLT.ThCtAB.MEAN', 'pLT.ThCtAB.MEAN', 'eLT.ThCtAB.MEAN',
    'aMT.ThCtAB.MEAN', 'iMT.ThCtAB.MEAN', 'cMT.ThCtAB.MEAN', 'pMT.ThCtAB.MEAN', 'eMT.ThCtAB.MEAN',
    'cLF.ThCtAB.MEAN', 'LT.ThCtAB.MEAN', 'cLFTC.ThCtAB.MEAN', 'LFTC.ThCtAB.MEAN',
    'cMF.ThCtAB.MEAN', 'MT.ThCtAB.MEAN', 'cMFTC.ThCtAB.MEAN', 'MFTC.ThCtAB.MEAN',
    
    'LT.ThCcAB.MEAN', 'MT.ThCcAB.MEAN', 'cMF.ThCcAB.MEAN', 'cLF.ThCcAB.MEAN',
)

comparisons = [
    ((4, ), (2, ), '000m', '012m'),
    ((4, ), (2, ), '000m', '024m'),
    ((3, 4), (1, 2), '000m', '012m'),
    ((3, 4), (1, 2), '000m', '024m'),
]

for kl in ("all", 1, 2, 3):
    df_tmp = df_sel_cmp.copy()
    
    # Select subject by KL-grade
    if kl == "all":
        pass
    else:
        df_tmp = df_tmp[df_tmp["V00XRKL"] == kl]
    
    for comparison in comparisons:
        cases_ctr, cases_aff, visit_base, visit_cont = comparison

        dict_res = defaultdict(list)

        # Select subject groups: control and case, baseline and continuation
        sel_ctr_base = df_tmp[df_tmp['CASE'].isin(cases_ctr) & (df_tmp['visit'] == visit_base)]
        sel_ctr_cont = df_tmp[df_tmp['CASE'].isin(cases_ctr) & (df_tmp['visit'] == visit_cont)]
        sel_aff_base = df_tmp[df_tmp['CASE'].isin(cases_aff) & (df_tmp['visit'] == visit_base)]
        sel_aff_cont = df_tmp[df_tmp['CASE'].isin(cases_aff) & (df_tmp['visit'] == visit_cont)]
        
        print(f"{kl}")
        print(len(sel_ctr_base), len(sel_aff_base))

        for feature_code in feature_codes:
            author_list = ("ours", analysis[0])
        
            for author in author_list:
                if f"{feature_code}_{author}" not in df_tmp.columns:
                    continue

                val_ctr_base = np.asarray(sel_ctr_base[f'{feature_code}_{author}'].tolist())
                val_ctr_cont = np.asarray(sel_ctr_cont[f'{feature_code}_{author}'].tolist())
                val_aff_base = np.asarray(sel_aff_base[f'{feature_code}_{author}'].tolist())
                val_aff_cont = np.asarray(sel_aff_cont[f'{feature_code}_{author}'].tolist())
                
                diff_ctr = val_ctr_base - val_ctr_cont
                diff_aff = val_aff_base - val_aff_cont

                X = [*diff_ctr, *diff_aff]
                y_true = [0, ] * len(diff_ctr) + [1, ] * len(diff_aff)

                X = np.asarray(X)
                y_true = np.asarray(y_true)
                sel = np.invert(np.isnan(X))
                X = X[sel][..., None]
                y_true = y_true[sel]

                clf = sklearn.linear_model.LogisticRegression(penalty='none',
                                                              class_weight='balanced',
                                                              solver='newton-cg')
                clf.fit(X, y_true)
                y_pred = clf.predict(X)

                cm = sklearn.metrics.confusion_matrix(y_true, y_pred)
                odds_ratio, pvalue = stats.fisher_exact(cm)

                if pvalue < 0.05:
                    print(feature_code, author)

                dict_res['feature_code'].append(feature_code)
                dict_res['author'].append(author)

                dict_res['odds_ratio'].append(odds_ratio)
                dict_res['pvalue'].append(f"{pvalue:0.3f}")
                dict_res['KL'] = kl

        df_res = pd.DataFrame.from_dict(dict_res)
        display(df_res)

        path_out = Path(f"/home/egor/Workspace/proj_scartan/results/t/"
                    f"csvs_{analysis[0]}_{analysis[1]}")
        path_out.mkdir(exist_ok=True, parents=True)

        str_ctr = "".join(str(e) for e in cases_ctr)
        str_aff = "".join(str(e) for e in cases_aff)

        df_res.to_csv(Path(
            path_out, f"{str_ctr}vs{str_aff}_{visit_base}-{visit_cont}-{kl}.csv"))